In [50]:
import pandas as pd
from datetime import datetime, timedelta, time
import json
# To ignore warnings
import warnings
warnings.filterwarnings("ignore")


# Preparing Emp Availability With Working Flag and Time Remaining

## 1. Collect Emp Availability Table and Prepare

In [25]:
path=r"00_Input\01_Emp_Availability_Initial.xlsx"
df=pd.read_excel(path,names=['Name', 'Roles', 'Responsibility', 'Time in', 'Time out'])

# Convert 'Time in' and 'Time out' to datetime first
df['Time in'] = pd.to_datetime(df['Time in'], format='%H:%M:%S')
df['Time out'] = pd.to_datetime(df['Time out'], format='%H:%M:%S')

# Filter out not-required roles and names
filtered_df = df[~df['Responsibility'].isin(['Technology', 'Office Work'])& ~df['Name'].str.contains('Available')]

# Display the filtered DataFrame
filtered_df.head(5)

,Name,Roles,Responsibility,Time in,Time out
0,Andrew (AJ) # Bowlen,Sales - Students,Lead Student,1900-01-01 08:30:00,1900-01-01 14:00:00
1,Analuisa Flores Teran,Sales - Students,Sales Floor/Cashier,1900-01-01 08:45:00,1900-01-01 11:00:00
5,Finn Broderick,Sales - Students,Sales Floor/Cashier,1900-01-01 08:45:00,1900-01-01 11:00:00
6,Ricardo Gomez,Sales - Students,Sales Floor/Cashier,1900-01-01 08:45:00,1900-01-01 12:00:00
8,Solymar Kneale,Sales - Students,Sales Floor/Cashier,1900-01-01 08:45:00,1900-01-01 10:30:00


## 2. Create full joined working flag table

In [26]:
new_rows = []
time_interval=30

first_start_time = filtered_df['Time in'].min()
max_end_time = filtered_df['Time out'].max()

# Get unique names from filtered_df
unique_names = filtered_df['Name'].unique()

# Iterate through each individual
for name in unique_names:
    start_time = first_start_time
    end_time = max_end_time
    
    # Generate 30 minute intervals
    while start_time < end_time:
        new_row = {
            'Name': name,
            'Start_time': start_time.time(),
            'End_time': (start_time + timedelta(minutes=time_interval)).time()
        }
        new_rows.append(new_row)
        start_time += timedelta(minutes=30)

# Create the new DataFrame
work_status_df = pd.DataFrame(new_rows)


def get_working_flag(name, start_time, end_time):
    # Get all working hours for the specific name
    employee_records = filtered_df[filtered_df['Name'] == name]
    
    # Iterate through all records for the employee
    for index, record in employee_records.iterrows():
        # Convert Time in and Time out to time objects
        time_in = record['Time in'].time()  # Use .time() to get the time object
        time_out = record['Time out'].time()  # Use .time() to get the time object

        # Check if the Start_time and End_time fall within the working hours
        if (time_in <= start_time < time_out) or (time_in < end_time <= time_out) or (start_time <= time_in and end_time >= time_out):
            return 1  # Working
    
    return 0  # Not working if none of the records match

# Add the 'Working Flag' column to work_status_df
work_status_df['Working Flag'] = work_status_df.apply(
    lambda row: get_working_flag(row['Name'], row['Start_time'], row['End_time']), axis=1
)

# Display the updated DataFrame
work_status_df.sample(5)

,Name,Start_time,End_time,Working Flag
511,Scott Hackney,14:00:00,14:30:00,1
575,Diego Navarro,16:00:00,16:30:00,1
629,Cassidy Sylves,13:00:00,13:30:00,0
528,Summer Sutton,12:30:00,13:00:00,0
144,Ethan Palmer,10:30:00,11:00:00,1


In [27]:
# # Filter only working emp times and rename columns
# greeter_priority_df= work_status_df[work_status_df['Working Flag']==1]
# greeter_priority_df.rename({"Name": "Employee"})
# greeter_priority_df.sample(5)

work_status_df= work_status_df[work_status_df['Working Flag']==1]
greeter_priority_df= work_status_df.copy()
work_status_copy_df= work_status_df.copy()

# Calculate remaining hours left
# Ensure all time columns are converted to strings in case they are of type datetime.time
greeter_priority_df['Start_time'] = greeter_priority_df['Start_time'].astype(str)
greeter_priority_df['End_time'] = greeter_priority_df['End_time'].astype(str)
work_status_copy_df['Start_time'] = work_status_copy_df['Start_time'].astype(str)
work_status_copy_df['End_time'] = work_status_copy_df['End_time'].astype(str)

# Function to calculate remaining working hours
def calculate_remaining_hours(employee, current_time, work_status_df):
    # Filter the work_status_copy_df for the specific employee and working hours
    employee_schedule = work_status_df[(work_status_df['Name'] == employee) & (work_status_df['Working Flag'] == 1)]
    
    remaining_time = 0
    current_time = datetime.strptime(current_time, '%H:%M:%S')  # Convert to datetime object
    
    for _, row in employee_schedule.iterrows():
        shift_start = datetime.strptime(row['Start_time'], '%H:%M:%S')
        shift_end = datetime.strptime(row['End_time'], '%H:%M:%S')
        
        if shift_start >= current_time:
            remaining_time += (shift_end - shift_start).total_seconds() / 3600  # Convert seconds to hours
    
    return remaining_time

# Add 'Remaining_hours_left' column to greeter_priority_df
greeter_priority_df['Remaining_hours_left'] = greeter_priority_df.apply(
    lambda row: calculate_remaining_hours(row['Name'], row['End_time'], work_status_copy_df), axis=1
)

# Display the updated dataframe
greeter_priority_df.head(5)

,Name,Start_time,End_time,Working Flag,Remaining_hours_left
0,Andrew (AJ) # Bowlen,08:30:00,09:00:00,1,5.0
1,Andrew (AJ) # Bowlen,09:00:00,09:30:00,1,4.5
2,Andrew (AJ) # Bowlen,09:30:00,10:00:00,1,4.0
3,Andrew (AJ) # Bowlen,10:00:00,10:30:00,1,3.5
4,Andrew (AJ) # Bowlen,10:30:00,11:00:00,1,3.0


In [29]:
# Testing

greeter_priority_df[greeter_priority_df['Name']=='Ana* Gonzalez']

,Name,Start_time,End_time,Working Flag,Remaining_hours_left
163,Ana* Gonzalez,10:00:00,10:30:00,1,3.0
164,Ana* Gonzalez,10:30:00,11:00:00,1,2.5
170,Ana* Gonzalez,13:30:00,14:00:00,1,2.0
171,Ana* Gonzalez,14:00:00,14:30:00,1,1.5
172,Ana* Gonzalez,14:30:00,15:00:00,1,1.0
173,Ana* Gonzalez,15:00:00,15:30:00,1,0.5
174,Ana* Gonzalez,15:30:00,16:00:00,1,0.0


In [30]:
greeter_priority_df['Name'].unique()

array(['Andrew (AJ) # Bowlen', 'Analuisa Flores Teran', 'Finn Broderick',
       'Ricardo Gomez', 'Solymar Kneale', 'Carson Turk', 'Dev Mahajan',
       'Ethan Palmer', 'Ana* Gonzalez', 'Marsailles Wesley',
       'Josiah Galvan-Lewis', 'Kaeden Stander', 'Hanah # Wilkins',
       'Kevin^ Jenkins', 'Luke* Ruhl', 'Joaquin Mendoza', 'Miguel Tanner',
       'Elijah Kodjak', 'Lauren* Koski', 'Cristian Andrade',
       'Lulu (Lyndsey)* Arterburn', 'AJ # Alkhamees', 'Joaquin Salinas',
       'Cassidy Jacobs', 'Mariana Cepeda', 'Scott Hackney',
       'Summer Sutton', 'Anahi Guitierrez', 'Diego Navarro',
       'Lohith Ramesh', 'Saaijeesh Sottalu Naresh', 'Cassidy Sylves',
       'Zadie Knapp', 'Serenity* Munoz', 'Elizabeth Jurry',
       'Madelyn* Ozdarski', 'Naman Talwar', 'Aaron* Dorrance',
       'Calvin # Levy', 'Ellia* Bono', 'Kayhaan Rashiq', 'Mali Chavez'],
      dtype=object)

# Create Shift Requrement

In [32]:
# Import emp_count_req Default Table
emp_count_req= pd.read_excel(r"00_Input\02_Emp_Count_Requirement.xlsx")
emp_count_req.head()

# Convert 'From_Time' and 'To_Time' to datetime objects
emp_count_req['From_Time'] = pd.to_datetime(emp_count_req['From_Time'], format='%H:%M:%S').dt.time
emp_count_req['To_Time'] = pd.to_datetime(emp_count_req['To_Time'], format='%H:%M:%S').dt.time

emp_count_req.head()

,From_Time,To_Time,Reg_Up_Needed,Reg_Down_Needed,Greeter_Up_Needed,Greeter_Down_Needed,Min_Total_Emp_Needed
0,07:30:00,08:00:00,3,3,0,0,6
1,08:00:00,08:30:00,3,3,0,0,6
2,08:30:00,09:00:00,3,3,1,1,8
3,09:00:00,09:30:00,3,3,1,1,8
4,09:30:00,10:00:00,3,3,1,1,8


In [34]:
work_status_df.head()

,Name,Start_time,End_time,Working Flag
0,Andrew (AJ) # Bowlen,08:30:00,09:00:00,1
1,Andrew (AJ) # Bowlen,09:00:00,09:30:00,1
2,Andrew (AJ) # Bowlen,09:30:00,10:00:00,1
3,Andrew (AJ) # Bowlen,10:00:00,10:30:00,1
4,Andrew (AJ) # Bowlen,10:30:00,11:00:00,1


In [39]:
emp_aval= work_status_df.copy()
emp_aval.rename(columns={'Start_time': 'Work_From', 'End_time': 'Work_To', 'Working Flag': 'Working_Flag'}, inplace=True)


# Create "total available employee" column

# Group By the From and To time of "Work Status Per Time" table and count the number of available employees
grouped_avl_by_time= emp_aval.groupby(['Work_From', 'Work_To']).agg({'Working_Flag': 'sum'}).reset_index()
grouped_avl_by_time.columns= ['Work_From', 'Work_To', 'Total_Avl_Emp']

# Join it with the emp_count_req table
emp_demand_check= pd.merge(emp_count_req, grouped_avl_by_time, how='left', left_on=['From_Time','To_Time'], right_on=['Work_From','Work_To'])
columns_needed= ['From_Time', 'To_Time', 'Reg_Up_Needed', 'Reg_Down_Needed', 'Greeter_Up_Needed', 'Greeter_Down_Needed', 'Min_Total_Emp_Needed', 'Total_Avl_Emp']
emp_demand_check= emp_demand_check[columns_needed]
emp_demand_check['Total_Avl_Emp'].fillna(0, inplace=True) # If no emp are available on a selected shift, make the availability zero. 

# Create an availability check flag and alert 
def alert_employee_shortage(emp_demand_check: pd.DataFrame):
    shortage= emp_demand_check[emp_demand_check['Availability_Check_Flag'] == False]
    if(len(shortage)==0):
        print("No shortage of employees for the whole day")
    else:
        print("ALERT: Employees are on shortage for the following time slots")
        print(shortage[['From_Time', 'To_Time', 'Min_Total_Emp_Needed', 'Total_Avl_Emp']])
    return

emp_demand_check['Availability_Check_Flag']= emp_demand_check['Min_Total_Emp_Needed']<= emp_demand_check['Total_Avl_Emp']
alert_employee_shortage(emp_demand_check)
display(emp_demand_check.sample(5))


ALERT: Employees are on shortage for the following time slots
   From_Time   To_Time  Min_Total_Emp_Needed  Total_Avl_Emp
0   07:30:00  08:00:00                     6            0.0
1   08:00:00  08:30:00                     6            0.0
2   08:30:00  09:00:00                     8            5.0
4   09:30:00  10:00:00                     8            7.0
18  16:30:00  17:00:00                    10            9.0
22  18:30:00  19:00:00                     7            0.0
23  19:00:00  19:30:00                     7            0.0


,From_Time,To_Time,Reg_Up_Needed,Reg_Down_Needed,Greeter_Up_Needed,Greeter_Down_Needed,Min_Total_Emp_Needed,Total_Avl_Emp,Availability_Check_Flag
16,15:30:00,16:00:00,5,3,1,1,10,13.0,True
22,18:30:00,19:00:00,2,3,1,1,7,0.0,False
3,09:00:00,09:30:00,3,3,1,1,8,8.0,True
9,12:00:00,12:30:00,5,3,1,1,10,10.0,True
20,17:30:00,18:00:00,2,3,1,1,7,13.0,True


# Greeter Assignment

In [42]:
greeter_assignment=emp_demand_check[['From_Time','To_Time','Greeter_Down_Needed','Greeter_Up_Needed']]
greeter_assignment.head()

,From_Time,To_Time,Greeter_Down_Needed,Greeter_Up_Needed
0,07:30:00,08:00:00,0,0
1,08:00:00,08:30:00,0,0
2,08:30:00,09:00:00,1,1
3,09:00:00,09:30:00,1,1
4,09:30:00,10:00:00,1,1


In [43]:
# Get unique names
unique_names = filtered_df['Name'].unique()

# Create a dictionary with names as keys and 0 as the initial value
greeter_shift_done_dict = {name: 0 for name in unique_names}

# Display the initialized dictionary
greeter_shift_done_dict


{'Andrew (AJ) # Bowlen': 0,
 'Analuisa Flores Teran': 0,
 'Finn Broderick': 0,
 'Ricardo Gomez': 0,
 'Solymar Kneale': 0,
 'Carson Turk': 0,
 'Dev Mahajan': 0,
 'Ethan Palmer': 0,
 'Ana* Gonzalez': 0,
 'Marsailles Wesley': 0,
 'Josiah Galvan-Lewis': 0,
 'Kaeden Stander': 0,
 'Hanah # Wilkins': 0,
 'Kevin^ Jenkins': 0,
 'Luke* Ruhl': 0,
 'Joaquin Mendoza': 0,
 'Miguel Tanner': 0,
 'Elijah Kodjak': 0,
 'Lauren* Koski': 0,
 'Cristian Andrade': 0,
 'Lulu (Lyndsey)* Arterburn': 0,
 'AJ # Alkhamees': 0,
 'Joaquin Salinas': 0,
 'Cassidy Jacobs': 0,
 'Mariana Cepeda': 0,
 'Scott Hackney': 0,
 'Summer Sutton': 0,
 'Anahi Guitierrez': 0,
 'Diego Navarro': 0,
 'Lohith Ramesh': 0,
 'Saaijeesh Sottalu Naresh': 0,
 'Cassidy Sylves': 0,
 'Zadie Knapp': 0,
 'Serenity* Munoz': 0,
 'Elizabeth Jurry': 0,
 'Madelyn* Ozdarski': 0,
 'Naman Talwar': 0,
 'Aaron* Dorrance': 0,
 'Calvin # Levy': 0,
 'Ellia* Bono': 0,
 'Kayhaan Rashiq': 0,
 'Mali Chavez': 0}

In [46]:
# Convert 'Start_time' and 'End_time' to time objects in the main dataframe
greeter_priority_df['Start_time'] = pd.to_datetime(greeter_priority_df['Start_time'], format='%H:%M:%S').dt.time
greeter_priority_df['End_time'] = pd.to_datetime(greeter_priority_df['End_time'], format='%H:%M:%S').dt.time

def assign_priority(df, filtered_df, greeter_shift_done_dict):
    # Initialize the priority column
    df['Priority'] = 0  # Initialize the Priority column with zeros
    
    # Group by Start_time and End_time
    for (start, end), group in df.groupby(['Start_time', 'End_time']):
        responsible_people = filtered_df[(filtered_df['Time in'] == start) & 
                                         (filtered_df['Time out'] == end) & 
                                         (filtered_df['Responsibility'] == 'Greeter')]
        
        if not responsible_people.empty:
            for index, row in responsible_people.iterrows():
                greeter = row['Name']
                if greeter in group['Name'].values:
                    df.loc[(df['Start_time'] == start) & (df['End_time'] == end) & (df['Name'] == greeter), 'Priority'] = 1
            
            remaining_group = group[~group['Name'].isin(responsible_people['Name'])]
            non_zero_hours_group = remaining_group[remaining_group['Remaining_hours_left'] > 0].sort_values(by='Remaining_hours_left', ascending=True)
            
            # Assign dense ranking starting from 2 for non-zero hours employees
            if not non_zero_hours_group.empty:
                non_zero_hours_group['Priority'] = non_zero_hours_group['Remaining_hours_left'].rank(method='dense', ascending=True).astype(int) + 1
                df.loc[non_zero_hours_group.index, 'Priority'] = non_zero_hours_group['Priority']
            
            # Employees with 0 hours remaining
            zero_hours_group = remaining_group[remaining_group['Remaining_hours_left'] == 0]
            if not zero_hours_group.empty:
                max_priority = non_zero_hours_group['Priority'].max() if not non_zero_hours_group.empty else 1
                zero_hours_group['Priority'] = max_priority + 1
                df.loc[zero_hours_group.index, 'Priority'] = zero_hours_group['Priority']
        else:
            group_with_non_zero_hours = group[group['Remaining_hours_left'] > 0].sort_values(by='Remaining_hours_left', ascending=True)
            group_with_zero_hours = group[group['Remaining_hours_left'] == 0]
            if not group_with_non_zero_hours.empty:
                group_with_non_zero_hours['Priority'] = group_with_non_zero_hours['Remaining_hours_left'].rank(method='dense', ascending=True).astype(int)
                df.loc[group_with_non_zero_hours.index, 'Priority'] = group_with_non_zero_hours['Priority']
            if not group_with_zero_hours.empty:
                max_priority = group_with_non_zero_hours['Priority'].max() if not group_with_non_zero_hours.empty else 1
                group_with_zero_hours['Priority'] = max_priority + 1
                df.loc[group_with_zero_hours.index, 'Priority'] = group_with_zero_hours['Priority']

    return df  # Return the modified DataFrame with the Priority column

In [47]:
def process_time_periods(greeter_priority_df, greeter_assignment, filtered_df, greeter_shift_done_dict):
    
    # Assign priorities before processing time periods
    greeter_priority_df = assign_priority(greeter_priority_df, filtered_df, greeter_shift_done_dict)

    time_periods = greeter_assignment[['From_Time', 'To_Time', 'Greeter_Up_Needed', 'Greeter_Down_Needed']].drop_duplicates()

    for idx, period in time_periods.iterrows():
        # Extract needed counts for upstairs and downstairs greeters
        up_needed = int(period['Greeter_Up_Needed'])
        down_needed = int(period['Greeter_Down_Needed'])

        current_period_data = greeter_priority_df[
            (greeter_priority_df['Start_time'] == period['From_Time']) &
            (greeter_priority_df['End_time'] == period['To_Time'])
        ]

        # Debug: Check current period data
        print(f"Current Period: {period['From_Time']} to {period['To_Time']}")
        print("Current Period Data:\n", current_period_data)

        # Sort employees by priority
        sorted_employees = current_period_data.sort_values('Priority')

        # Assign upstairs greeters based on needed count
        upstairs_greeters = sorted_employees.head(up_needed)['Name'].tolist()
        print("Upstairs Greeters Assigned:", upstairs_greeters)

        # Filter out already assigned upstairs greeters for downstairs assignments
        remaining_employees = sorted_employees[~sorted_employees['Name'].isin(upstairs_greeters)]

        # Assign downstairs greeters based on needed count
        downstairs_greeters = remaining_employees.head(down_needed)['Name'].tolist() if down_needed > 0 else []
        print("Downstairs Greeters Assigned:", downstairs_greeters)
        print("-----------------------------------------------------------------------------")

        # Update the greeter_assignment DataFrame with assigned greeters
        greeter_assignment.at[idx, 'Upstairs Greeter'] = upstairs_greeters[0] if upstairs_greeters else None
        greeter_assignment.at[idx, 'Downstairs Greeter'] = downstairs_greeters[0] if downstairs_greeters else None

        # Update shift counts for assigned employees
        for employee in upstairs_greeters + downstairs_greeters:
            if employee:  # Only update for actual assignments
                greeter_shift_done_dict[employee] += 1

        # Update the priorities for assigned greeters
        for employee in upstairs_greeters + downstairs_greeters:
            if employee:  # Only update for actual assignments
                greeter_priority_df.loc[greeter_priority_df['Name'] == employee, 'Priority'] = greeter_priority_df['Priority'].max() + 1

    return greeter_assignment, greeter_shift_done_dict

# Example call to the function
greeter_assignment, greeter_shift_done_dict = process_time_periods(
    greeter_priority_df, 
    greeter_assignment, 
    filtered_df, 
    greeter_shift_done_dict
)

# Output results
greeter_assignment.head(50)

Current Period: 07:30:00 to 08:00:00
Current Period Data:
 Empty DataFrame
Columns: [Name, Start_time, End_time, Working Flag, Remaining_hours_left, Priority]
Index: []
Upstairs Greeters Assigned: []
Downstairs Greeters Assigned: []
-----------------------------------------------------------------------------
Current Period: 08:00:00 to 08:30:00
Current Period Data:
 Empty DataFrame
Columns: [Name, Start_time, End_time, Working Flag, Remaining_hours_left, Priority]
Index: []
Upstairs Greeters Assigned: []
Downstairs Greeters Assigned: []
-----------------------------------------------------------------------------
Current Period: 08:30:00 to 09:00:00
Current Period Data:
                      Name Start_time  End_time  Working Flag  \
0    Andrew (AJ) # Bowlen   08:30:00  09:00:00             1   
20  Analuisa Flores Teran   08:30:00  09:00:00             1   
40         Finn Broderick   08:30:00  09:00:00             1   
60          Ricardo Gomez   08:30:00  09:00:00             1   

,From_Time,To_Time,Greeter_Down_Needed,Greeter_Up_Needed,Upstairs Greeter,Downstairs Greeter
0,07:30:00,08:00:00,0,0,None,None
1,08:00:00,08:30:00,0,0,None,None
2,08:30:00,09:00:00,1,1,Analuisa Flores Teran,Finn Broderick
3,09:00:00,09:30:00,1,1,Solymar Kneale,Ricardo Gomez
4,09:30:00,10:00:00,1,1,Ethan Palmer,Carson Turk
5,10:00:00,10:30:00,1,1,Marsailles Wesley,Ana* Gonzalez
6,10:30:00,11:00:00,1,1,Kaeden Stander,Andrew (AJ) # Bowlen
7,11:00:00,11:30:00,1,1,Kevin^ Jenkins,Luke* Ruhl
8,11:30:00,12:00:00,1,1,Miguel Tanner,Joaquin Mendoza
9,12:00:00,12:30:00,1,1,Lauren* Koski,Elijah Kodjak


# Register Allocation

In [48]:
shift_req_df=emp_demand_check[['From_Time', 'To_Time', 'Reg_Up_Needed', 'Reg_Down_Needed']]
shift_req_df.head()

,From_Time,To_Time,Reg_Up_Needed,Reg_Down_Needed
0,07:30:00,08:00:00,3,3
1,08:00:00,08:30:00,3,3
2,08:30:00,09:00:00,3,3
3,09:00:00,09:30:00,3,3
4,09:30:00,10:00:00,3,3


In [59]:
work_status_df= greeter_priority_df[greeter_priority_df['Working Flag']==1]
work_status_df.sample(2)

,Name,Start_time,End_time,Working Flag,Remaining_hours_left,Priority
163,Ana* Gonzalez,10:00:00,10:30:00,1,3.0,15
473,Cassidy Jacobs,15:00:00,15:30:00,1,3.0,37


In [61]:
# Function to convert the string of list to list in the greeter assignment table
def extract_list_from_string(string):
    return string.replace("'",'').replace("[",'').replace("]",'').split(",")

# Prepare intial variables
current_RUs= []
current_RDs= []
store_open_time= time(8, 30) #shift_req_df['From_Time'].min()
store_close_time= shift_req_df['From_Time'].max()
time_delta= timedelta(minutes=30)

curr_time= store_open_time

while (curr_time < store_close_time):
    curr_time_plus30= (datetime.combine(datetime.today(), curr_time) + time_delta).time()
    print(f"Allocating registers between {curr_time} to {curr_time_plus30} ...")

    # Working Employees
    all_working_emp_list= work_status_df[(work_status_df['Start_time']== curr_time) & (work_status_df['End_time']== curr_time_plus30)]['Name'].to_list()
    greeter_emp_list= extract_list_from_string(greeter_assignment[(greeter_assignment['From_Time']== curr_time) & (greeter_assignment['To_Time']== curr_time_plus30)]['Upstairs Greeter'].to_list()[0]) + extract_list_from_string(greeter_assignment[(greeter_assignment['From_Time']== curr_time) & (greeter_assignment['To_Time']== curr_time_plus30)]['Downstairs Greeter'].to_list()[0])

    # Baseed on the workring and greeter list, calculate the retined register list
    retained_RUs=[]
    retained_RDs= []

    for emp in current_RUs:
        if (emp not in all_working_emp_list):
            # EMP shift got over
            continue
        if (emp in greeter_emp_list):
            # EMP moved to greeter
            continue
        else:
            retained_RUs.append(emp)
    
    for emp in current_RDs:
        if (emp not in all_working_emp_list):
            # EMP shift got over
            continue
        if (emp in greeter_emp_list):
            # EMP moved to greeter
            continue
        else:
            retained_RDs.append(emp)

    # Find how many new registers are needed
    needed_RU_count, needed_RD_count= shift_req_df[(shift_req_df['From_Time']== curr_time) & (shift_req_df['To_Time']== curr_time_plus30)][['Reg_Up_Needed','Reg_Down_Needed']].values[0]
    RU_retained_count= len(retained_RUs)
    RD_retained_count= len(retained_RDs)
    new_RU_needed_count= needed_RU_count-RU_retained_count
    new_RD_needed_count= needed_RD_count-RD_retained_count

    # Create the prriority table. Use the same table to assign both RU and RD
    priority_table= work_status_df[(work_status_df['Start_time']== curr_time) & (work_status_df['End_time']== curr_time_plus30) & (work_status_df['Name'] not in retained_RUs+retained_RDs)]
    priority_table['RU_Priority'] = priority_table['Remaining_hours_left'].rank(method='min', ascending=False).astype(int)
    priority_table.sort_values(by='RU_Priority', inplace=True)
    priority_table.reset_index(drop=True, inplace=True)
    display(priority_table.head())

    # Pick the new registers for both RU and RD
    print('new_RU_needed_count', new_RU_needed_count)
    print('new_RD_needed_count', new_RD_needed_count)
    new_RU_assigned= priority_table['Name'][:new_RU_needed_count].tolist()
    new_RD_assigned= priority_table['Name'][new_RU_needed_count: new_RU_needed_count+new_RD_needed_count].tolist()
    print('new_RU_assigned', new_RU_assigned)
    print('new_RD_assigned', new_RD_assigned)
    
    # Should not break this, rather increment the start time by 30mins so that the while loop continues
    break


Allocating registers between 08:30:00 to 09:00:00 ...


,Name,Start_time,End_time,Working Flag,Remaining_hours_left,Priority,RU_Priority
0,Andrew (AJ) # Bowlen,08:30:00,09:00:00,1,5.0,17,1
1,Ricardo Gomez,08:30:00,09:00:00,1,3.0,11,2
2,Solymar Kneale,08:30:00,09:00:00,1,2.5,10,3
3,Analuisa Flores Teran,08:30:00,09:00:00,1,2.0,8,4
4,Finn Broderick,08:30:00,09:00:00,1,2.0,9,4


new_RU_needed_count 3
new_RD_needed_count 3
new_RU_assigned ['Andrew (AJ) # Bowlen', 'Ricardo Gomez', 'Solymar Kneale']
new_RD_assigned ['Analuisa Flores Teran', 'Finn Broderick']


In [ ]:
# TO DO

#- Fix time remaining calculation logic. 
# 	- Person doing multiple shifts with breaks in between not working.
# 	- Time remaining should be from the current time.

# - Test with matching emp count req and emp availability

# - Currently ignoring if "Greeter" as initial role assigned 
# - Take the reamining available emp not in both greeter and register and allocate them to salesfloor
# - Seperate UP and DOWN logic